# Template Matching Tutorial Notebook 



Sections:
1. Libraries
2. Template construction 
3. Template visualization
3. Matching / cross-correlation window 
4. Plotting


![Data](./seismicitymap.png)


Repo: https://dasway.ess.washington.edu/mora/index.html

## 0. Libraries


In [ ]:
import os, re, glob
from datetime import datetime, timezone, timedelta
from typing import List
from scipy.signal import butter
from datetime import datetime, timezone, timedelta
import time
from templatematching_cc import *
from getanalisisfiles import *
from template_maker_w import *

In [ ]:
# -------- settings you likely only tweak here --------
base_year_dir = "/data/fast0/rainier50Hz/2023/"
# propagate this value into the MODULE

set_base_year_dir(base_year_dir)

# where to write templates and CC results
templates_dir = "/data/data2/workshop/temp_files"
cc_base       = "/data/data2/workshop/cc_results"
os.makedirs(templates_dir, exist_ok=True)
os.makedirs(cc_base, exist_ok=True)

# AUGUST window for templates (USGS events):
tmpl_start_dt = datetime(2023, 8, 27, 10, 0)
tmpl_end_dt   = datetime(2023, 8, 27, 10, 30)

# MATCHING window (example: full Oct 27 UTC)

match_start = "2023-08-27_09.00"
match_end   = "2023-08-27_11.00"

# DAS + filter
chan_min, chan_max = 1000,1200
template_size = 5.0
fs = 50
samples_per_file = int(60 * fs)
b, a = butter(2, (2.0, 10.0), 'bp', fs=fs)

# filename pattern 
rx = re.compile(r"rainier_50Hz_(\d{4}-\d{2}-\d{2})_(\d{2})\.(\d{2})\.(\d{2})_UTC\.h5$")

## 1. Template construction


In [ ]:
#clean this away
#def yday(dt: datetime) -> int:
#    return dt.timetuple().tm_yday

#def path_for_time(t_utc: datetime) -> str:
#    """Return full path to file that covers t_utc, based on minute-aligned filenames."""
#    t_utc = t_utc.replace(tzinfo=timezone.utc)
#    # files are 1-minute long and named ..._HH.MM.00_UTC.h5
#    t0 = t_utc.replace(second=0, microsecond=0)
#    day_dir = os.path.join(base_year_dir, f"{yday(t0):03d}")
#    fn = f"rainier_50Hz_{t0.strftime('%Y-%m-%d_%H.%M')}.00_UTC.h5"
#    return os.path.join(day_dir, fn)

In [ ]:
# ---------------- A) TEMPLATES (AUGUST) ----------------
events = search(starttime=tmpl_start_dt,
                endtime=tmpl_end_dt,
                latitude=46.879967, longitude=-121.726906,
                maxradius=35/111.32)
event_df = get_summary_data_frame(events).sort_values("time")
print(f"[templates] usgs events in august: {len(event_df)}")

found_files = []
original_dates_fixed = []

miss = 0
hit  = 0
for _, row in event_df.iterrows():
    # event time as naive UTC datetime
    t_evt = row["time"]  # should already be UTC (pandas/obspy)
    if isinstance(t_evt, str):
        t_evt = datetime.strptime(
            t_evt.replace("_"," ").replace(".",":"),
            "%Y-%m-%d %H:%M:%S"
        ).replace(tzinfo=timezone.utc)
    else:
        t_evt = pd.Timestamp(t_evt).to_pydatetime().replace(tzinfo=timezone.utc)

    p = path_for_time(t_evt)
    if os.path.isfile(p):
        found_files.append(p)
        # template_maker2 expects 'YYYY-MM-DD HH:MM:SS.%f' for the event time
        original_dates_fixed.append(t_evt.strftime("%Y-%m-%d %H:%M:%S.%f"))
        hit += 1
    else:
       # print(f"No raw file for event: {t_evt.strftime('%Y-%m-%d_%H.%M.%S')}")
        miss += 1

if hit > 0:
    process_files_to_cut(
        found_files, original_dates_fixed,
        base_year_dir, templates_dir,
        chan_min, chan_max, template_size
    )
else:
    print("[templates] no templates cut (no august raw found)")

template_list = glob.glob(os.path.join(templates_dir, "*"))
#print(f"[templates] templates found on disk: {len(template_list)}")

## 2. Plotting 1 template


In [ ]:
#path of one example, need to make it prettier\
p = "/data/data2/workshop/temp_files/2023-08-27_10-10-23.740000.h5"  
with h5py.File(p, "r") as h5:
    cand = ["data","phase","strain_rate","strainrate","/data","/phase","/strain_rate","/strainrate"]
    ds = next((h5[k] for k in cand if k in h5), None)
    if ds is None:
        names=[]; h5.visititems(lambda n,o: names.append(n) if isinstance(o,h5py.Dataset) and not names else None)
        ds = h5[names[0]]
    A  = ds[()]
    fs = float(ds.attrs.get("sampling_rate", ds.attrs.get("Fs", 50.0)))
M = A if A.ndim==2 and A.shape[0]>=A.shape[1] else (A.T if A.ndim==2 else A[:,None])
t = np.arange(M.shape[0])/fs
plt.imshow(M.T, aspect="auto", vmin =- 0.1, vmax = 0.1, extent=[t.min(), t.max(), 0, M.shape[1]], cmap="seismic")
plt.xlabel("Time (s)"); plt.ylabel("Channel"); plt.colorbar(label="phase"); plt.tight_layout()
plt.title('Raw data')
plt.show()


## 2. Matching / cross-correlation window (serial, no multiprocessing)


In [ ]:
file_list = build_file_list_julian(match_start, match_end)
#print(f"[matching] raw files: {len(file_list)} | templates: {len(template_list)}")

cc_out = os.path.join(cc_base, f"CC_{int(template_size)}sec-tem_{match_start}-{match_end}")
os.makedirs(cc_out, exist_ok=True)

if not file_list:
    print("[matching] no raw files in the chosen window")
elif not template_list:
    print("[matching] no templates available")
else:
    t0 = time.perf_counter()
    process_files_dos(
        file_list, template_list,
        chan_min, chan_max, (chan_max - chan_min),
        samples_per_file, b, a, cc_out
    )
    dt = time.perf_counter() - t0

## 3. Timestamps and plotting


In [ ]:
file_list = build_file_list_julian(match_start, match_end)
print(len(file_list))
ts_h5_dir= '/data/data2/workshop/cc_results/timestamps'
h5_path = create_timestamps_h5(file_list, ts_h5_dir)

In [ ]:
Template_folder_1 = '/data/data2/workshop/cc_results/CC_5sec-tem_2023-08-27_09.00-2023-08-27_11.00/2023-08-27_10-10-23.740000'

time_tag = f"{match_start}-{match_end}"  # e.g. "2023-08-27_09.00-2023-08-27_12.30"

CC_DIR  = os.path.join(
    cc_base,
    f"CC_{int(template_size)}sec-tem_{time_tag}",
    Template_folder_1,
)
H5_FILE = h5_path  # from create_timestamps_h5

print("[CC_DIR ]", CC_DIR)
print("[H5_FILE]", H5_FILE)

t_abs, t_abs_aligned, cc, cc_files = load_cc_and_time(
    CC_DIR,
    H5_FILE,
    template_size_sec=template_size,
)

print(f"[align] N = {len(cc)}, range: {t_abs_aligned[0]} → {t_abs_aligned[-1]}")


In [ ]:
#plotting
# --- load timestamps and align with CC ---
with h5py.File(h5_path, "r") as h5:
    ts_raw = h5["timestamps"][:]          # microseconds since epoch

# make sure lengths match
N = min(len(cc), len(ts_raw))
cc_plot = np.asarray(cc[:N], float)
ts_raw = ts_raw[:N]

# convert to Python datetime for plotting
epoch = datetime(1970, 1, 1)
time_utc = np.array(
    [epoch + timedelta(microseconds=int(t)) for t in ts_raw]
)

print("N =", N)
print("time range:", time_utc[0], "→", time_utc[-1])

# --- simple plot: CC vs time ---
plt.figure(figsize=(12, 4))
plt.plot(time_utc, abs(cc_plot), lw=0.4)
plt.xlabel("Time (UTC)")
plt.ylabel("Correlation (CC)")
plt.title("Results for cc for 1 template")
plt.tight_layout()
plt.show()


In [ ]:
Template_folder_2 = '/data/data2/workshop/cc_results/CC_5sec-tem_2023-08-27_09.00-2023-08-27_11.00/2023-08-27_10-26-15.030000'

time_tag = f"{match_start}-{match_end}"  # e.g. "2023-08-27_09.00-2023-08-27_12.30"

CC_DIR  = os.path.join(
    cc_base,
    f"CC_{int(template_size)}sec-tem_{time_tag}",
    Template_folder_2,
)
H5_FILE = h5_path  # from create_timestamps_h5

print("[CC_DIR ]", CC_DIR)
print("[H5_FILE]", H5_FILE)

t_abs, t_abs_aligned, cc, cc_files = load_cc_and_time(
    CC_DIR,
    H5_FILE,
    template_size_sec=template_size,
)

print(f"[align] N = {len(cc)}, range: {t_abs_aligned[0]} → {t_abs_aligned[-1]}")

In [ ]:
with h5py.File(h5_path, "r") as h5:
    ts_raw = h5["timestamps"][:]          # microseconds since epoch

# make sure lengths match
N = min(len(cc), len(ts_raw))
cc_plot = np.asarray(cc[:N], float)
ts_raw = ts_raw[:N]

# convert to Python datetime for plotting
epoch = datetime(1970, 1, 1)
time_utc = np.array(
    [epoch + timedelta(microseconds=int(t)) for t in ts_raw]
)

print("N =", N)
print("time range:", time_utc[0], "→", time_utc[-1])

# --- simple plot: CC vs time ---
plt.figure(figsize=(12, 4))
plt.plot(time_utc, abs(cc_plot), lw=0.4)
plt.xlabel("Time (UTC)")
plt.ylabel("Correlation (CC)")
plt.title("Results for cc for 2 template")
plt.tight_layout()
plt.show()

In [ ]:
# plotting
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

THRESH = 0.117          # threshold on |CC|
TOL_SEC = 6.0         # time tolerance to merge hits into one event

# --- load timestamps and align with CC ---
with h5py.File(h5_path, "r") as h5:
    ts_raw = h5["timestamps"][:]          # microseconds since epoch

# make sure lengths match
N = min(len(cc), len(ts_raw))
cc_plot = np.asarray(cc[:N], float)
ts_raw = ts_raw[:N]

# convert to Python datetime for plotting
epoch = datetime(1970, 1, 1)
time_utc = np.array(
    [epoch + timedelta(microseconds=int(t)) for t in ts_raw]
)

print("N =", N)
print("time range:", time_utc[0], "→", time_utc[-1])

# --- find samples over threshold (absolute value) ---
cc_abs = np.abs(cc_plot)
idx_hits = np.where(cc_abs >= THRESH)[0]

print(f"\nNumber of samples with |CC| >= {THRESH}: {len(idx_hits)}")

if len(idx_hits) > 0:
    hit_times = time_utc[idx_hits]
    hit_vals  = cc_abs[idx_hits]

    # --- cluster hits within TOL_SEC and keep max in each cluster ---
    clusters = []
    start = 0
    for i in range(1, len(hit_times)):
        dt = (hit_times[i] - hit_times[i-1]).total_seconds()
        if dt > TOL_SEC:
            clusters.append((start, i))   # [start, i)
            start = i
    clusters.append((start, len(hit_times)))  # last cluster

    event_times = []
    event_vals = []
    for s, e in clusters:
        # indices within this cluster
        local_max_idx = s + np.argmax(hit_vals[s:e])
        event_times.append(hit_times[local_max_idx])
        event_vals.append(hit_vals[local_max_idx])

    event_times = np.array(event_times)
    event_vals  = np.array(event_vals)

    print(f"\nNumber of unique events (tolerance {TOL_SEC} s): {len(event_times)}")
    for t, v in zip(event_times, event_vals):
        print(f"  {t}  |CC| = {v:.3f}")
else:
    event_times = np.array([])
    event_vals  = np.array([])

# --- plot CC vs time with threshold + unique-event markers ---
plt.figure(figsize=(12, 4))
plt.plot(time_utc, cc_abs, lw=0.4, label="|CC|")
plt.axhline(THRESH, color="r", linestyle="--", label=f"threshold = {THRESH}")

if event_times.size > 0:
    plt.scatter(event_times, event_vals, s=25, color="r", label="unique events")

plt.xlabel("Time (UTC)")
plt.ylabel("|Correlation (CC)|")
plt.title("CC vs Time (unique peaks with 6 s tolerance)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ---- IRIS / ComCat query for the same time window ----
client = Client("IRIS")

start = UTCDateTime(time_utc[0])
end   = UTCDateTime(time_utc[-1])

LAT, LON = 46.879967, -121.726906
MAXR_KM = 35.0
MAXR_DEG = MAXR_KM / 111.32

cat = client.get_events(
    starttime=start,
    endtime=end,
    latitude=LAT,
    longitude=LON,
    maxradius=MAXR_DEG,
)

cat_times = np.array([ev.origins[0].time.datetime for ev in cat])
cat_mags  = np.array([ev.magnitudes[0].mag if ev.magnitudes else np.nan for ev in cat])

print(f"[IRIS] catalog events in window: {len(cat_times)}")
for t, m in zip(cat_times, cat_mags):
    if np.isnan(m):
        print(f"  {t}  M=?")
    else:
        print(f"  {t}  M={m:.2f}")

# ---- compare DAS events to catalog (time tolerance) ----
TOL_SEC = 10  # tolerance for matching catalog events, e.g. ±20 s
tol = np.timedelta64(int(TOL_SEC * 1000), "ms")

das64 = event_times.astype("datetime64[ms]")
cat64 = cat_times.astype("datetime64[ms]") if len(cat_times) else np.array([])

matched_idx = []
new_idx = []

for i, t in enumerate(das64):
    if cat64.size and np.any(np.abs(cat64 - t) <= tol):
        matched_idx.append(i)
    else:
        new_idx.append(i)

print(f"\nDAS unique events: {len(event_times)}")
print(f"  matched to catalog (±{TOL_SEC}s): {len(matched_idx)}")
print(f"  no catalog match: {len(new_idx)}")

for i in matched_idx:
    print(f"  MATCHED: {event_times[i]}  |CC|={event_vals[i]:.3f}")

for i in new_idx:
    print(f"  NEW:     {event_times[i]}  |CC|={event_vals[i]:.3f}")